# Backend pipeline for GUI

In [1]:
import numpy as np
import pandas as pd
import os
import sys
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import pickle as pickle
from src.Log import Reformat
from src.Utils import DataUtil
from src.Trainer import CaseDataSet
from src.Trainer import Regressor
from src.Trainer import Classifier
from src.Preprocessing import Extraction
from src.Preprocessing import Preprocess

from src.Backend import DataProcessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

In [5]:
logs = DataUtil.read_files("../../data/SimulatedLogs", ".json")

In [27]:
log_pd = {}
for file_name, data in logs.items():
    log_pd[file_name] = DataProcessing.generate_trace(data, time_column="endDate", case_column="processInstanceKey", selected_columns=["flowNodeId", "endDate", "processInstanceKey"])

In [29]:
t1 = log_pd['bpmn1-log']

In [2]:
with open("../../presets/t2.pkl", 'rb') as f:
    data = pickle.load(f)

In [3]:
d1 = data["traces_dict"]["bpmn1-log_trace"]

In [4]:
t1 = CaseDataSet.CaseDataset(d1, feature_list=["flowNodeId", "LapseTime"], label="RemTime", encoding="all")

In [5]:
def split_data(log, train=0.64, val=0.16):
    data_size = log.shape[0]
    training_set = log[:int(train * data_size)]
    validation_set = log[int(train * data_size): int(train * data_size) + int(val * data_size)]
    test_set = log[int(train * data_size) + int(val * data_size): ]
    return training_set, validation_set, test_set


def norm_remtime(train, val, test, columns=["RemTime", "LapseTime"]):
    for column in columns:
        max_value = train[column].explode(column).max()
        train.loc[:, column] = train[column] / max_value
        val.loc[:, column] = val[column] / max_value
        test.loc[:, column] = test[column] / max_value 
    return train, val, test

In [6]:
train, val, test = split_data(d1)
train, val, test = norm_remtime(train, val, test)

In [7]:
train_t1 = CaseDataSet.CaseDataset(train, feature_list=["flowNodeId", "LapseTime"], label="RemTime", encoding="all")
val_t1 = CaseDataSet.CaseDataset(val, feature_list=["flowNodeId", "LapseTime"], label="RemTime", encoding="all")

In [8]:
r1 = Regressor.XGBTrainer(train_t1, val_t1, "../../models/XGB/")

ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1 and the array at index 1 has size 2